In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00


In [ ]:
import logging
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from sklearn.metrics import accuracy_score, f1_score

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the dataset (we'll use the test split for evaluation)
dataset = load_dataset('zeroshot/twitter-financial-news-sentiment', split='validation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/859k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [ ]:
# Shuffle the dataset and slice it
sliced_dataset = dataset.shuffle(seed=42).select(range(1000))

In [ ]:
sliced_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [ ]:
# Sentiment mapping: 0 = negative, 1 = positive, 2 = neutral
sentiment_mapping = {
    0: "negative",
    1: "positive",
    2: "neutral"
}

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Your GPU supports bfloat16: accelerate training with bf16=True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Define the text-generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
# Function to generate sentiment
def generate_sentiment(text):
    # Construct the prompt
    prompt = f"Determine the sentiment of the financial news as negative, neutral or positive: {text}"

    # Run the text generation
    result = pipe(f"<s>[INST] {prompt} [/INST]")

    # Extract the generated text
    generated_text = result[0]['generated_text']

    # Extract the text after [/INST]
    return extract_after_inst(generated_text)

In [ ]:
# Function to extract the part after [/INST]
def extract_after_inst(generated_text):
    parts = generated_text.split('[/INST]')
    return parts[1].strip() if len(parts) > 1 else generated_text

In [ ]:
# Function to check sentiment in the model's output
def check_sentiment(output):
    output = output.lower()
    if "negative" in output:
        return "negative"
    elif "neutral" in output:
        return "neutral"
    elif "positive" in output:
        return "positive"
    else:
        return "neutral"  # Default to "neutral" if no keyword found

In [ ]:
# List to store predictions and actual labels
y_true = []
y_pred = []

# Iterate over the dataset and generate predictions
for sample in dataset:
    # Get the text and actual label
    text = sample['text']
    actual_label = sentiment_mapping[sample['label']]

    # Run the model to get predicted sentiment
    generated_text = generate_sentiment(text)
    predicted_sentiment = check_sentiment(generated_text)

    # Append the true and predicted labels for evaluation
    y_true.append(actual_label)
    y_pred.append(predicted_sentiment)

    # Optional: Print the generated result and sentiment
    print(f"Generated text: {generated_text}")
    print(f"True Sentiment: {actual_label}, Predicted Sentiment: {predicted_sentiment}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Generated text: Based on the tweet "Ally Financial pulls outlook," the sentiment of the financial news is likely negative. The use of the word "pulls" and the phrase "outlook" suggests that Ally Financial is revising its previous forecast or prediction, which is often a negative occurrence in the financial industry. Therefore, the sentiment of the news can be classified as negative.
True Sentiment: negative, Predicted Sentiment: negative
Generated text: Based on the tweet "Dell, HPE targets trimmed on compute headwinds," the sentiment of the financial news is NEGATIVE.

The word "trimmed" in this context implies a reduction or downsizing of targets, which is typically a negative development in the financial world. The mention of "compute headwinds" also suggests that the companies are facing challenges in the computing sector, which could further contribute to a negative sentiment. Overall, the tone of the tweet is negative towards Dell and HPE.
True Sentiment: negative, Predicted Sent

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Generated text: Based on the information provided in the financial news article, the sentiment can be classified as neutral. nobody is happy about the downgrade, but neither is anyone particularly upset. The price target has been slashed, which could be seen as a negative development, but the rating has only been downgraded from outperform to neutral, which is a more neutral assessment than a downgrade from a buy to a sell, for example. Therefore, the sentiment of the financial news can be classified as neutral.
True Sentiment: negative, Predicted Sentiment: negative
Generated text: Based on the information provided, the sentiment of the financial news can be classified as neutral. nobody.

The news of Clovis Oncology being downgraded to "in line" from "outperform" at Evercore ISI indicates that the analyst's expectations for the company's stock performance have been revised to be more in line with the current market conditions. This does not necessarily mean that the company is perfor

KeyboardInterrupt: 

In [ ]:
def generate_and_classify_batch(batch):
    texts = batch['text']
    actual_labels = [sentiment_mapping[label] for label in batch['label']]

    # Generate predictions for the batch
    generated_texts = pipe([f"<s>[INST] Determine the sentiment of the financial news as negative, neutral or positive: {text} [/INST]" for text in texts])

    # Flattening the nested lists of generated texts
    if isinstance(generated_texts[0], list):
        generated_texts = [item for sublist in generated_texts for item in sublist]

    # Extract and classify sentiment from generated texts
    predicted_sentiments = [check_sentiment(extract_after_inst(result['generated_text'])) for result in generated_texts]

    print("Batch completed.")

    return {'y_true': actual_labels, 'y_pred': predicted_sentiments}

# Apply the transformation using map with batch_size
batch_size = 64
result = sliced_dataset.map(generate_and_classify_batch, batched=True, batch_size=batch_size)

# Extract true and predicted labels for evaluation
y_true = result['y_true']
y_pred = result['y_pred']


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Batch completed.
Batch completed.
Batch completed.
Batch completed.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.


In [ ]:
# Evaluate the model using accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.4250
F1 Score: 0.4241
